# How To: Compute likelihood and chi2

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
from smodels.statistics.simplifiedLikelihoods import LikelihoodComputer, Data

## How to compute the likelihood and chi2 from the number of events

In [3]:
# If the number of observed events, the number of expected background events,
# its error and the number of signal events and its error are known, the likelihood
# for the signal (assuming a truncated gaussian distribution for the background and signal uncertainties)
# can be computed as:
m=Data ( observed=5, backgrounds=4.2, covariance=0.71**2, third_moment=None, nsignal=.1 )
comp=LikelihoodComputer ( m )
print ('likelihood (mu=1): %1.3e' %comp.likelihood(mu=1.0) )
print ('likelihood (mu=100): %1.3e' %comp.likelihood(mu = 100. ) )
print ('chi2: %1.3e' %comp.chi2())

likelihood (mu=1): 9.395e-02
likelihood (mu=100): 2.043e-03
chi2: 9.646e-02


In [4]:
## It is however advisable to work with (negative) log likelihoods, instead of likelihood values:
print ('nll(mu=1): %1.3f' %comp.likelihood(mu=1.0, return_nll=True ) )
print ('nll(mu=100): %1.3f' %comp.likelihood(mu = 100., return_nll=True ) )

nll(mu=1): 2.365
nll(mu=100): 6.193


## How to compute the likelihood and chi2 from a theory prediction

In [5]:
# In most cases one wants to compute the likelihood and chi2 for a given theory prediction computed by SModelS.
# Below we generate theory predictions and compute the likelihood and chi2 values for them
# First we import those parts of smodels that are needed for this exercise
#(We will assume the input is a SLHA file. For LHE files, use the lheDecomposer instead)
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.installation import installDirectory
from smodels.base.physicsUnits import fb, GeV
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.base import smodelsLogging
smodelsLogging.setLogLevel('error')


In [6]:
# Define the SLHA input file name
filename="inputFiles/slha/gluino_squarks.slha"
model = Model(BSMparticles = BSMList, SMparticles = SMList)
model.updateParticles(inputFile=filename)

In [7]:
# Load the database, do the decomposition and compute theory predictions:
# (Look at the theory predictions HowTo to learn how to compute theory predictions)
database = Database("official")
database.selectExpResults(analysisIDs = [ "ATLAS-SUSY-2018-31"] )
topList = decomposer.decompose(model, sigmacut = 0.03 * fb, massCompress=True, invisibleCompress=True,minmassgap = 5* GeV)
allThPredictions = theoryPredictionsFor(database, topList)

In [8]:
# For each theory prediction, compute the corresponding likelihood and chi2 values
# (This is only possible for efficiency map-type results):
for theoryPred in allThPredictions:
    expID = theoryPred.analysisId()
    dataType = theoryPred.dataType()
    #Compute the likelihood and chi2:
    theoryPred.computeStatistics()
    print ("\nExperimental Result: %s (%s-type)" %(expID,dataType)) #Result ID and type
    print ("Theory prediction xsec = ",theoryPred.xsection) #Signal xsection*efficiency*BR
    if dataType in [ 'efficiencyMap', 'combined' ]:
        theoryPred.computeStatistics()
        print('nll_BSM, nll_SM, nll_min =', theoryPred.likelihood( return_nll=True ), \
              theoryPred.lsm( return_nll=True ), theoryPred.lmax( return_nll=True) )

    else:
        print ("(likelihood not available)")



Experimental Result: ATLAS-SUSY-2018-31 (combined-type)
Theory prediction xsec =  4.14E-06 [pb]
nll_BSM, nll_SM, nll_min = 3.919977287643161 3.76268804500569 3.762688044977277

Experimental Result: ATLAS-SUSY-2018-31 (upperLimit-type)
Theory prediction xsec =  2.87E-04 [pb]
(likelihood not available)
